In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
import matplotlib.image as mpimg
from collections import Counter

In [2]:
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')

In [3]:
train_data.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [4]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7613 entries, 0 to 7612
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        7613 non-null   int64 
 1   keyword   7552 non-null   object
 2   location  5080 non-null   object
 3   text      7613 non-null   object
 4   target    7613 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 297.5+ KB


Predict whether a given tweet is about a real distaster or not.

In [27]:
tokenizer = keras.preprocessing.text.Tokenizer(char_level=True)

In [35]:
vocabulary = Counter()
for tweet in train_data['text']:
    #tweet = tf.strings.regex_replace(tweet, b'[^a-zA-Z]', b' ')
    tweet = tf.strings.split(tweet)
    vocabulary.update(list(tweet.numpy()))

In [38]:
len(vocabulary.keys())

31924

It looks like we have 31924 words in our dictionary. Lets take the top 10,000 of them and used them in our model.

In [39]:
vocab_size = 10000
truncated_vocabulary = [ word for word, count in vocabulary.most_common()[:vocab_size]]

In [40]:
#preprocess to replace each word with its ID (index)
words = tf.constant(truncated_vocabulary)
word_ids = tf.range(len(truncated_vocabulary), dtype=tf.int64)
vocab_init = tf.lookup.KeyValueTensorInitializer(words, word_ids)
num_oov_buckets = 1000
table = tf.lookup.StaticVocabularyTable(vocab_init,num_oov_buckets)

In [41]:
table.lookup(tf.constant([b'this movie was faaaaaaantastic'.split()]))

<tf.Tensor: shape=(1, 4), dtype=int64, numpy=array([[   24,   337,    21, 10770]], dtype=int64)>

In [43]:
def encode_words(X_batch, y_batch):
    return table.lookup(X_batch), y_batch

In [56]:
train_set = train_data[['text','target']]
target = train_set.pop('target')
dataset = tf.data.Dataset.from_tensor_slices((train_set.values, target.values)).batch(32)

Lets make sure that the dataset converted correctly. We are going to iterate over the dataset and print it's features and target

In [57]:
for feat, targ in dataset.take(5):
  print ('Features: {}, Target: {}'.format(feat, targ))

Features: [[b'Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all']
 [b'Forest fire near La Ronge Sask. Canada']
 [b"All residents asked to 'shelter in place' are being notified by officers. No other evacuation or shelter in place orders are expected"]
 [b'13,000 people receive #wildfires evacuation orders in California ']
 [b'Just got sent this photo from Ruby #Alaska as smoke from #wildfires pours into a school ']
 [b'#RockyFire Update => California Hwy. 20 closed in both directions due to Lake County fire - #CAfire #wildfires']
 [b'#flood #disaster Heavy rain causes flash flooding of streets in Manitou, Colorado Springs areas']
 [b"I'm on top of the hill and I can see a fire in the woods..."]
 [b"There's an emergency evacuation happening now in the building across the street"]
 [b"I'm afraid that the tornado is coming to our area..."]
 [b'Three people died from the heat wave so far']
 [b'Haha South Tampa is getting flooded hah- WAIT A SECOND I LIVE IN SOUTH TAMPA W

In [58]:
train_set = dataset.map(encode_words).prefetch(1)

In [60]:
embed_size = 128
model = keras.models.Sequential([
    keras.layers.Embedding(vocab_size + num_oov_buckets, embed_size, input_shape=[None]),
    keras.layers.GRU(128, return_sequences=True),
    keras.layers.GRU(128),
    keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss= 'binary_crossentropy', optimizer='Adam', metrics=['accuracy'])
history = model.fit(train_set, epochs=25)

Epoch 1/25
238/238 [==============================] - 3s 13ms/step - loss: 0.6888 - accuracy: 0.5706
Epoch 2/25
238/238 [==============================] - 3s 13ms/step - loss: 0.6610 - accuracy: 0.6073
Epoch 3/25
238/238 [==============================] - 3s 13ms/step - loss: 0.5951 - accuracy: 0.6708
Epoch 4/25
238/238 [==============================] - 3s 13ms/step - loss: 0.5701 - accuracy: 0.6857
Epoch 5/25
238/238 [==============================] - 3s 13ms/step - loss: 0.5581 - accuracy: 0.6953
Epoch 6/25
238/238 [==============================] - 3s 13ms/step - loss: 0.5542 - accuracy: 0.6962
Epoch 7/25
238/238 [==============================] - 3s 13ms/step - loss: 0.5508 - accuracy: 0.7002
Epoch 8/25
238/238 [==============================] - 3s 13ms/step - loss: 0.5494 - accuracy: 0.7035
Epoch 9/25
238/238 [==============================] - 3s 13ms/step - loss: 0.5483 - accuracy: 0.7075
Epoch 10/25
238/238 [==============================] - 3s 13ms/step - loss: 0.5473 - accura

In [69]:
#using a pretrained embedding
import tensorflow_hub as hub
model = keras.Sequential([
    hub.KerasLayer('https://tfhub.dev/google/tf2-preview/nnlm-en-dim50/1',
        dtype=tf.string, input_shape=[], output_shape=[50]),
        keras.layers.Dense(16,activation='relu'),
        keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [71]:
# This is broken
batch_size = 32
train_set = dataset.batch(batch_size).prefetch(1)
tweet_model = model.fit(train_set, epochs=5)

Epoch 1/5


ValueError: in user code:

    C:\Users\joshu\anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:806 train_function  *
        return step_function(self, iterator)
    C:\Users\joshu\anaconda3\lib\site-packages\tensorflow_hub\keras_layer.py:228 call  *
        result = f()
    C:\Users\joshu\anaconda3\lib\site-packages\tensorflow\python\saved_model\load.py:509 _call_attribute  **
        return instance.__call__(*args, **kwargs)
    C:\Users\joshu\anaconda3\lib\site-packages\tensorflow\python\eager\def_function.py:780 __call__
        result = self._call(*args, **kwds)
    C:\Users\joshu\anaconda3\lib\site-packages\tensorflow\python\eager\def_function.py:814 _call
        results = self._stateful_fn(*args, **kwds)
    C:\Users\joshu\anaconda3\lib\site-packages\tensorflow\python\eager\function.py:2828 __call__
        graph_function, args, kwargs = self._maybe_define_function(args, kwargs)
    C:\Users\joshu\anaconda3\lib\site-packages\tensorflow\python\eager\functio